In [1]:
import heapq
import os
import tracemalloc
import time
class Puzzle:
    def __init__(self, state, g=0, h=0,parent=None, action=None):
        self.state = state
        self.g = g
        self.parent = parent
        self.action = action
        self.heuristic = h
        
    def __lt__(self, other):
        return self.g < other.g
                
    def __len__(self):
        return len(self.state)
    def heuristic(self):
        self.heuristic = calculateHorizontal(self.state) + calculateVertical(self.state)                

# TO CHECK THE PROBLEM IS SOLVABLE OR NOT
def calculate_NumberOfInversion(board):
    row = []
    for i in range(len(board)):
        for j in range(len(board)):
            row.append(board[i][j])
            
    count = 0
    for i in range(len(row)):
        for j in range(i + 1, len(row)):
            if (row[j] < row[i] and row[j] != 0):
                count = count + 1
    return count
            

def rowOfBlankPos(board):
    for i in range(len(board)):
        for j in range(len(board[i])):
            if (board[i][j] == 0):
                return i
            
def Is_solvable(node):
    NOI = calculate_NumberOfInversion(node.state)
    row_blankpos = rowOfBlankPos(node.state) + 1
    print(row_blankpos)
    size = len(node.state)
    if (size % 2 != 0 and NOI % 2 == 0):
        return True
    elif (NOI % 2 == 0 and row_blankpos % 2 == 0) or (NOI % 2 != 0 and row_blankpos % 2 != 0):
        return True
    
    return False

# IMPLEMENT UNINFORMED COST SEARCH
def UCS(initial_state, target_state):
    node = initial_state
    frontier = [(node.g, initial_state)]
    visited = {initial_state: node}

    while frontier:
        _,node = heapq.heappop(frontier)
        
        if is_goal(node.state, target_state):
            return node
        
        for child in expand(initial_state, node):
            if is_goal(child.state, target_state):
                return child
            if child not in visited or child.g < visited[child].g:
                visited[child] = child  
                heapq.heappush(frontier, (visited[child], child))
    return None

def expand(initial_state, node):
    s = node.state
    list = []
    
    for action in actions(s):
        _s = result(s, action)
        cost = node.g + 1
        _heuristic = calculateVertical(_s) + calculateHorizontal(_s)
        child = Puzzle(state=_s, g=cost, h=_heuristic, parent=node, action=action) 
        list.append(child)

    return list
    
def is_goal(board, target_state):
    return board == target_state

def findBlankSpace(board):
    for i in range(len(board)):
        for j in range(len(board)):
            if (board[i][j] == 0):
                return i, j

def actions(board):
    blank_i, blank_j = findBlankSpace(board)
    actions = []
    if (blank_i > 0):
        actions.append("up")
        
    if (blank_i < len(board) - 1):
        actions.append("down")
        
    if (blank_j < len(board) - 1):
        actions.append("right")
        
    if (blank_j > 0):
        actions.append("left")
        
    return actions
    

def result(_board, action):
    blank_i, blank_j = findBlankSpace(_board)
    board = [row[:] for row in _board]
    #Move up
    if action == "up":
        board[blank_i][blank_j], board[blank_i - 1][blank_j] = board[blank_i - 1][blank_j], board[blank_i][blank_j]
    #Move down
    if action == "down":
        board[blank_i][blank_j], board[blank_i + 1][blank_j] = board[blank_i + 1][blank_j], board[blank_i][blank_j]  
    
    #Move left
    if action == "left":
        board[blank_i][blank_j], board[blank_i][blank_j - 1] = board[blank_i][blank_j - 1], board[blank_i][blank_j]

    #Move right
    if action == "right":
        board[blank_i][blank_j], board[blank_i][blank_j + 1] = board[blank_i][blank_j + 1], board[blank_i][blank_j]
    
    return board

# INSTALL INVERSION DISTANCE
def calculateVertical(board):
    row = []
    for i in range(len(board)):
        for j in range(len(board)):
            row.append(board[i][j])
            
    invcountVertical = 0
    for i in range(len(row)):
        for j in range(i+1, len(row)):
            if (row[i] > row[j] and row[j] != 0):
                invcountVertical = invcountVertical + 1
                
    return (invcountVertical / 3 + invcountVertical % 3)

def calculateHorizontal(board):
    col = []
    for i in range(len(board)):
        for j in range(len(board)):
            col.append(board[i][j])
            
    invcountHorizontal = 0
    for i in range(len(col)):
        for j in range(len(col)):
            if (col[i] > col[j] and col[j] != 0):
                invcountHorizontal = invcountHorizontal + 1
                
    return (invcountHorizontal / 3 + invcountHorizontal % 3)

# IMPLEMENT A STAR ALGORITHM
def AStarAlgorithm(initial_state, target_state):
    node = Puzzle(initial_state)
    node_heuristic = node.heuristic
    frontier = [(node_heuristic + node.g, initial_state)]
    g_score = {initial_state: 0}
    f_score = {initial_state: initial_state.heuristic}

    while frontier:
        _, node = heapq.heappop(frontier)
      
        if is_goal(node.state, target_state):
          return node
      
        for child in expand(initial_state, node):
            if is_goal(child.state, target_state):
                return child
            child.f = child.g + child.heuristic
            if (child not in g_score or node.g + node.heuristic < child.g + child.heuristic):
              g_score[child] = node.g
              f_score[child] = node.heuristic + node.g
              heapq.heappush(frontier, (f_score[child], child))

    return None

def inputPuzzle(board, size):
    for i in range(size):
        row = []
        for j in range(size):
            row.append(int(input(f"Enter element for row {i+1}, column {j+1}: ")))
        board.append(row)
    
def checkValidPuzzle(board, size):
    row = []
    for i in range(size):
        for j in range(size):
            row.append(board[i][j])
            
    for i in range(0, len(row)):
        for j in range(i+1, len(row)):
            if (row[i] == row[j] or row[i] < 0 or row[i] > 9 or 0 not in row):
                return False
        return True


def solveUCS(initial_state, target_state):
    if (Is_solvable(initial_state) == True):
        tracemalloc.start()
        start_time = time.time()
        solution = UCS(initial_state, target_state)
        end_time = time.time()
        execute_time = end_time - start_time
        # Print the solution, run time and consumed memory
        if solution:
            path= []
            current = solution
            while current:
                path.append(current.state)
                current = current.parent

            path.reverse()
            print("Solution path: ")
            for board in path:
                for row in board:
                    print(row)
                print()

            print("Execution time: ", round(execute_time, 3) * 1000, " milliseconds")
            print("Consumed memory: ", round(tracemalloc.get_traced_memory()[0] / (1024 * 1024), 3), " MB")
         
        tracemalloc.stop()
    else: 
        print("No solution found.")

    
def solveAStar(initial_state, target_state):
    if (Is_solvable(initial_state) == True):
        tracemalloc.start()
        start_time = time.time()
        solution = AStarAlgorithm(initial_state, target_state)
        end_time = time.time()
        execute_time = end_time - start_time
        # Print the solution, run time and consumed memory
        if solution:
            path = []
            current = solution
            while current:
                path.append(current.state)
                current = current.parent

            path.reverse()
            print("Solution path: ")
            for board in path:
                for row in board:
                    print(row)
                print()

            print("Execution time: ", round(execute_time, 3) * 1000, " milliseconds")
            print("Consumed memory: ", round((tracemalloc.get_traced_memory()[0] / (1024 * 1024)), 3), " MB")
        
        tracemalloc.stop()
    else: 
        print("No solution found.")

    
def create_target_state(result, size):
    index = 1
    for i in range(size):
        row = []
        for j in range(size):
            if (i != size - 1 or j != size - 1):
              element = index
              row.append(element)
              index = index + 1
            else:
              row.append(0)
        result.append(row)
            
def solveNPuzzle(board, SIZE):

    target_state = []
    create_target_state(target_state, SIZE)
    #os.system('clear')  #To clear screen
    print("1. Create your own puzzle")
    print("2. Use available puzzle in program")
    choice2 = int(input("Enter your choice: "))
    if (choice2 == 1):
        inputPuzzle(board, SIZE)
        if (checkValidPuzzle(board, SIZE)):
            initial_state = Puzzle(board, 0)
            print("Choose the Search Algorithm")
            print("1. Uniform-cost search (UCS)")
            print("2. Graph-search A* with INVERSION DISTANCE")
            choice3 = int(input("Enter your choice: "))
            if (choice3 == 1):
                solveUCS(initial_state, target_state)
                return
            elif (choice3 == 2):
                solveAStar(initial_state, target_state)
                return
            else:
                print("Invalid choice!")
                return
        else: 
            print("Inappropriate puzzle!")
    elif (choice2 == 2):
        if (SIZE == 3):
            board = [[2, 0, 5], [8, 3, 4], [1, 7, 6]]
        elif (SIZE == 4):
            board = [[1, 3, 4, 8], [6, 2, 10, 7], [5, 9, 12, 0], [13, 14, 11, 15]]
        elif (SIZE == 6):
            board = [[1, 2, 3, 4, 5, 6], [0, 8, 9, 10, 11, 12], [7, 20, 14, 15, 17, 18],
                     [13, 19, 21, 16, 23, 24],[25, 26, 27, 22, 28, 30], [31, 32, 33, 34, 29, 35]]
        initial_state = Puzzle(board, 0)
        print("Choose the Search Algorithm")
        print("1. Uniform-cost search (UCS)")
        print("2. Graph-search A* with INVERSION DISTANCE")
        choice3 = int(input("Enter your choice: "))
        if (choice3 == 1):
            solveUCS(initial_state, target_state)
            return
        elif (choice3 == 2):
            solveAStar(initial_state, target_state)
            return
        else:
            print("Invalid choice!")
            return
    else:
        print("Invalid choice!")
        
if __name__ == "__main__":
    SIZE = 0
    print("------------------ N PUZZLE GAME ------------------")
    print() 
    print("We have 3 types of N Puzzle Game. Please choose the size.")
    print("1. 8 Puzzle")
    print("2. 15 Puzzle")
    print("3. 35 Puzzle")
    choice1 = int(input("Enter your choice: "))
    board = []
    while True:
        if (choice1 == 1):
            SIZE = 3
            solveNPuzzle(board, SIZE)
            break
        elif (choice1 == 2):
            SIZE = 4
            solveNPuzzle(board, SIZE)
            break
        elif(choice1 == 3):
            SIZE = 6
            solveNPuzzle(board, SIZE)
            break
        else:
            print("Invalid choice")
    


------------------ N PUZZLE GAME ------------------

We have 3 types of N Puzzle Game. Please choose the size.
1. 8 Puzzle
2. 15 Puzzle
3. 35 Puzzle
